In [211]:
import pandas as pd
import numpy as np
import gc

from xgboost import XGBClassifier
from xgboost import plot_importance

from sklearn.utils import resample

from sklearn.model_selection import train_test_split, cross_val_score

from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, roc_auc_score
#from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import LabelEncoder

In [212]:
seed=123
#today = pd.to_datetime('2018-10-28')
#poly = PolynomialFeatures(interaction_only=True,include_bias = False)


#df_majority=TRAIN[TRAIN['Loan_Status']=='Y']
#df_minority=TRAIN[TRAIN['Loan_Status']=='N']
#n=len(df_minority)

#df_majority_downsampled = resample(df_majority,replace=False,n_samples=n,random_state=seed) 

#TRAIN = pd.concat([df_majority_downsampled, df_minority])

In [213]:
TRAIN = pd.read_csv("traindataimputed.csv")
TEST = pd.read_csv("test.csv")
TEST['Loan_Status']=np.nan
TRAIN=TRAIN.append(TEST).reset_index(drop=True)

In [198]:
categcolumns=[]
ordinalcolumns=['Property_Area','Loan_Amount_Term','LoanAmount','CoapplicantIncome','ApplicantIncome','Dependents']
binarycolumns=['Married','Gender','Education','Self_Employed','Credit_History','Loan_Status']

In [214]:
def labelencode( df ):
    
    df=pd.DataFrame(LabelEncoder().fit_transform(df.dropna())).set_index(df.dropna().index.values)
    
    return df

for col in binarycolumns:
    TRAIN[col]=labelencode(TRAIN[col])
    
    
    
TRAIN['Property_Area'].replace({'Rural':0,'Semiurban':1,'Urban':2},inplace=True)
TRAIN['Dependents'].replace({'3+':3},inplace=True)
TRAIN['Dependents']=TRAIN['Dependents'].astype(float)
TRAIN['ApplicantIncome']=np.log(TRAIN['ApplicantIncome'])
TRAIN['CoapplicantIncome']=np.log(TRAIN['CoapplicantIncome'])

C:\Users\sebastian\Anaconda3\lib\site-packages\ipykernel_launcher.py:15: RuntimeWarning: divide by zero encountered in log
  from ipykernel import kernelapp as app
C:\Users\sebastian\Anaconda3\lib\site-packages\ipykernel_launcher.py:16: RuntimeWarning: divide by zero encountered in log
  app.launch_new_instance()


In [215]:
X=TRAIN.dropna(subset=['Loan_Status']).drop(['Loan_Status','Loan_ID'],axis=1)
Y=TRAIN['Loan_Status'].dropna()

X_train, X_test, Y_train, Y_test = train_test_split(X,Y)

model=XGBClassifier(missing=np.nan)
model=model.fit(X,Y)
predictions = model.predict(X_test)


FeatureImportance=pd.DataFrame(list(zip(X.columns,model.feature_importances_)),columns=['Feature','Importance']).sort_values(by=['Importance'],ascending=False).reset_index(drop=True)
FeatureImportance
#X=df3[FeatureImportance['Feature'].head(30)]

C:\Users\sebastian\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


,Feature,Importance
0,CoapplicantIncome,0.395699
1,ApplicantIncome,0.215054
2,LoanAmount,0.167742
3,Credit_History,0.066667
4,Loan_Amount_Term,0.062366
5,Property_Area,0.036559
6,Dependents,0.023656
7,Education,0.017204
8,Married,0.015054
9,Gender,0.000000


In [216]:
predictions = model.predict(TRAIN[pd.isnull(TRAIN['Loan_Status'])].drop(['Loan_Status','Loan_ID'],axis=1))

C:\Users\sebastian\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [217]:
finaldata=TRAIN[pd.isnull(TRAIN['Loan_Status'])].drop(['Loan_Status','Loan_ID'],axis=1)
finaldata['Loan_Status']=predictions
finaldata=finaldata.join(TRAIN.Loan_ID,how='left')
finaldata['Loan_Status'].replace({0:'N',1.0:'Y'},inplace=True)
finaldata.to_csv('finaldata.csv')

In [209]:


    
print(accuracy_score(Y_test,predictions))
print(cross_val_score(model, X, Y, cv=5))
print(np.mean(cross_val_score(model, X, Y, cv=5)))
print(classification_report(Y_test, predictions))
print(roc_auc_score(Y_test,predictions))


0.8441558441558441


C:\Users\sebastian\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\Users\sebastian\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\Users\sebastian\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\Users\sebastian\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Return

[0.80645161 0.75       0.79508197 0.85245902 0.81147541]
0.8030936012691697
             precision    recall  f1-score   support

        0.0       0.85      0.53      0.66        43
        1.0       0.84      0.96      0.90       111

avg / total       0.85      0.84      0.83       154

0.7494238424470984


C:\Users\sebastian\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\Users\sebastian\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\Users\sebastian\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\Users\sebastian\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Return